In [1]:
import sys
sys.path.append("./uncertainty_ensemble/")

In [10]:
from uncertainty_bootstrap_subsample import UncertaintyEnsembleRegressor
from sklearn.ensemble import BaggingRegressor
from lshiftml.feature_utils.parallel import get_features_in_parallel
from rascal.representations import SphericalInvariants as SOAP
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species
from ase.io import read
import json
import numpy as np
from loader.loader import load_data
from skcosmo.model_selection import atom_groups_by_frame
from sklearn.linear_model import Ridge

In [3]:
train_structures, test_structures, Ytrain, Ytest = load_data("/ssd/scratch/kellner/COSMO_project/make_tensor_data/train_tensor/CSD-3k+S546_shift_tensors.xyz",\
                                                                                    "/ssd/scratch/kellner/COSMO_project/make_tensor_data/test_tensor/CSD-500+104-7_shift_tensors.xyz",random_subsample_train=200,random_subsample_test=200,selected_species=1)

with open("/ssd/scratch/kellner/ShiftML-Light/src/lshiftml/models/RR_model_data/RR_this_work_models/hypers/1_hypers.json","r") as fg:
                hypers = json.load(fg)


Xtrain = get_features_in_parallel(train_structures,SOAP,hypers)
Xtest = get_features_in_parallel(test_structures,SOAP,hypers)

train_groups = atom_groups_by_frame(train_structures)

In [11]:
model = BaggingRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=False,bootstrap_features=False,max_features=0.75,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain)

BaggingRegressor(base_estimator=Ridge(alpha=0.001), bootstrap=False,
                 max_features=0.75, n_estimators=8, n_jobs=4)

In [15]:
model = EnsembleRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=False,bootstrap_features=False,max_features=0.75,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain)#,groups=train_groups)
#mask = model.rescale(Xtrain,Ytrain,Xtest,Ytest)

KeyboardInterrupt: 

In [16]:
from sklearn.model_selection import GroupShuffleSplit

In [20]:
splitter = GroupShuffleSplit(n_splits = 2, train_size=1.)

In [21]:
splitter.split(np.arange(100),np.arange(100))

<generator object BaseShuffleSplit.split at 0x7f1f216349e0>

In [23]:
import sys
sys.path.append("./uncertainty_ensemble/")

from uncertainty_bootstrap_subsample import UncertaintyEnsembleRegressor
from sklearn.ensemble import BaggingRegressor
from lshiftml.feature_utils.parallel import get_features_in_parallel
from rascal.representations import SphericalInvariants as SOAP
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species
from ase.io import read
import json
import numpy as np
from loader.loader import load_data
from skcosmo.model_selection import atom_groups_by_frame
from sklearn.linear_model import Ridge


atomic_species = 1
n_jobs=4
n_estimators = 8

train_structures, test_structures, Ytrain, Ytest = load_data("/ssd/scratch/kellner/COSMO_project/make_tensor_data/train_tensor/CSD-3k+S546_shift_tensors.xyz",\
                                                                                    "/ssd/scratch/kellner/COSMO_project/make_tensor_data/test_tensor/CSD-500+104-7_shift_tensors.xyz",random_subsample_train=200,random_subsample_test=200,selected_species=1)

with open("/ssd/scratch/kellner/ShiftML-Light/src/lshiftml/models/RR_model_data/RR_this_work_models/hypers/1_hypers.json","r") as fg:
                hypers = json.load(fg)


Xtrain = get_features_in_parallel(train_structures,SOAP,hypers)
Xtest = get_features_in_parallel(test_structures,SOAP,hypers)

train_groups = atom_groups_by_frame(train_structures)

#subsample feat
print("-------- subsample feat --------")
model = UncertaintyEnsembleRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=False,bootstrap_features=False,max_samples=0.9999999,max_features=0.75,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain,groups=train_groups)
mask = model.rescale(Xtrain,Ytrain,Xtest,Ytest)

#subsample samples

print("-------- subsample samples --------")
model = UncertaintyEnsembleRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=False,bootstrap_features=False,max_samples=0.75,max_features=1.,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain,groups=train_groups)
mask = model.rescale(Xtrain,Ytrain,Xtest,Ytest)

#subsample sample + feat
print("-------- subsample samples + feat --------")
model = UncertaintyEnsembleRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=False,bootstrap_features=False,max_samples=0.75,max_features=0.75,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain,groups=train_groups)
mask = model.rescale(Xtrain,Ytrain,Xtest,Ytest)

#bootstrap feat
print("-------- bootstrap feat --------")
model = UncertaintyEnsembleRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=False,bootstrap_features=True,max_samples=0.9999999,max_features=1.,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain,groups=train_groups)
mask = model.rescale(Xtrain,Ytrain,Xtest,Ytest)

#bootstrap samples
print("-------- bootstrap samples --------")
model = UncertaintyEnsembleRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=True,bootstrap_features=False,max_samples=1.,max_features=1.,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain,groups=train_groups)
mask = model.rescale(Xtrain,Ytrain,Xtest,Ytest)


#bootstrap sample + feat
print("-------- bootstrap samples + feat--------")
model = UncertaintyEnsembleRegressor(base_estimator=Ridge(alpha=0.001),n_estimators=n_estimators,bootstrap=True,bootstrap_features=True,max_samples=1.,max_features=1.,n_jobs=n_jobs)
model.fit(Xtrain,Ytrain,groups=train_groups)
mask = model.rescale(Xtrain,Ytrain,Xtest,Ytest)


-------- subsample feat --------
RMSE test: 0.71

                 RMSE train: 0.19

                 NLL without rescale: 5.78

                 NLL with rescale(a): 5.78

                 NLL with resacle(a) small M correction: 10.03

                 NLL with rescale(a,b,g): 1.17

                 NLL best: 0.35

                 NLL worst: 1.07

                 no rescale coeff: 0.00

                 rescale coeff (a): 0.00

                 rescale coeff (a,b,g): 0.00

                 alpha rescale(a): 1.00 
 
                 a,b,g: 1.00, 1.00, 1.00

-------- subsample samples --------
RMSE test: 0.70

                 RMSE train: 0.30

                 NLL without rescale: 3.19

                 NLL with rescale(a): 4.49

                 NLL with resacle(a) small M correction: 8.55

                 NLL with rescale(a,b,g): 5.55

                 NLL best: 0.34

                 NLL worst: 1.07

                 no rescale coeff: 0.00

                 rescale coeff (a): 0.0

RMSE test: 0.48

                 RMSE train: 0.43

                 NLL without rescale: 5.33

                 NLL with rescale(a): 0.69

                 NLL with resacle(a) small M correction: 0.71

                 NLL with rescale(a,b,g): 0.64

                 NLL best: -0.05

                 NLL worst: 0.68

                 rescale coeff (a): 0.00

                 rescale coeff (a,b,g): 4.61

                 alpha rescale(a): 3.43 
 
                 a,b,g: 1.88, 0.38, 0.02

